In [1]:
# TODO(steve): build test cases for this functionality spike (TDD method!)

In [2]:
import datetime
import urllib.request
from bs4 import BeautifulSoup

class SecurityPriceData:
    
    def __init__(self, name: str, date: datetime.datetime, open: float, high: float,
                 low: float, close: float, adj_close: float, volume: int=0):
        self.name = name
        self.date = date
        self.open = float(open.replace(',', ''))
        self.high = float(high.replace(',', ''))
        self.low = float(low.replace(',', ''))
        self.close = float(close.replace(',', ''))
        self.adj_close = float(adj_close.replace(',', ''))
        self.volume = int(volume.replace(',', ''))
    
    def __str__(self):
        s = f'name => {self.name}\n'
        s += f'date => {self.date}\n'
        s += f'open => {self.open}\n'
        s += f'high => {self.high}\n'
        s += f'low => {self.low}\n'
        s += f'close => {self.close}\n'
        s += f'adj_close => {self.adj_close}\n'
        s += f'volume => {self.volume}'
        return s

In [3]:
def get_security_price_data(ticker: str, date: datetime.datetime):
    url = r'https://finance.yahoo.com/quote/{ticker}/history?p={ticker}'
    req = urllib.request.Request(url.replace('{ticker}', ticker))
    with urllib.request.urlopen(req) as response:
        the_page = response.read().decode('utf-8')

    parsed_html = BeautifulSoup(the_page)
    data_table = parsed_html.body.find('table', attrs={'data-test': 'historical-prices'})
    data_table = data_table.find('tbody')

    # Currently returns the first one in the table
    for row in data_table.children:
        values = [col.next_element.text for col in row.children if col.find('span')]
        values[0] = datetime.datetime.strptime(values[0], '%b %d, %Y')
        if values[0] == date:
            s = SecurityPriceData(ticker, *values)
            return s
    
    return None

In [5]:
dt = datetime.datetime(2019, 8, 19)
s = get_security_price_data('GOOG', dt)
print(s)

name => GOOG
date => 2019-08-19 00:00:00
open => 1190.09
high => 1206.99
low => 1190.09
close => 1198.45
adj_close => 1198.45
volume => 1231600
